In [1]:
import GPy
import numpy as np

from GPy.kern import Kern
from GPy.kern.src.stationary import Stationary

In [2]:
observations = np.array([[-1.], [0.], [1.]])

In [4]:
class Matern52withDerivatives(Stationary):
    """
    Matern 5/2 kernel:

    .. math::

       k(r) = \sigma^2 (1 + \sqrt{5} r + \\frac53 r^2) \exp(- \sqrt{5} r)
    """
    def __init__(self, input_dim, variance=1., lengthscale=None, ARD=False, active_dims=None, name='Mat52'):
        super(Matern52withDerivatives, self).__init__(input_dim, variance, lengthscale, ARD, active_dims, name)

    def to_dict(self):
        """
        Convert the object into a json serializable dictionary.

        Note: It uses the private method _save_to_input_dict of the parent.

        :return dict: json serializable dictionary containing the needed information to instantiate the object
        """
        input_dict = super(Matern52withDerivatives, self)._save_to_input_dict()
        input_dict["class"] = "GPy.kern.Matern52"
        return input_dict


    @staticmethod
    def _build_from_input_dict(kernel_class, input_dict):
        useGPU = input_dict.pop('useGPU', None)
        return Matern52(**input_dict)

    def K_of_r(self, r):
        return self.variance*(1+np.sqrt(5.)*r+5./3*r**2)*np.exp(-np.sqrt(5.)*r)


    def dK_dr(self, r):
        return self.variance*(10./3*r -5.*r -5.*np.sqrt(5.)/3*r**2)*np.exp(-np.sqrt(5.)*r)

    def dK2_drdr(self, r):
        return self.variance * (10. / 3 - 10. * np.sqrt(5.) * r + 25. / 3 * r ** 2 + 5. * np.sqrt(5.) * r ** 3 - 25. / 3 * r ** 4) * np.exp(-np.sqrt(5.) * r)

    def Gram_matrix(self, F, F1, F2, F3, lower, upper):
        """
        Return the Gram matrix of the vector of functions F with respect to the RKHS norm. The use of this function is limited to input_dim=1.
    
        :param F: vector of functions
        :type F: np.array
        :param F1: vector of derivatives of F
        :type F1: np.array
        :param F2: vector of second derivatives of F
        :type F2: np.array
        :param F3: vector of third derivatives of F
        :type F3: np.array
        :param lower,upper: boundaries of the input domain
        :type lower,upper: floats
        """
        assert self.input_dim == 1
        def L(x,i):
            return(5*np.sqrt(5)/self.lengthscale**3*F[i](x) + 15./self.lengthscale**2*F1[i](x)+ 3*np.sqrt(5)/self.lengthscale*F2[i](x) + F3[i](x))
        n = F.shape[0]
        G = np.zeros((n,n))
        for i in range(n):
            for j in range(i,n):
                G[i,j] = G[j,i] = integrate.quad(lambda x : L(x,i)*L(x,j),lower,upper)[0]
        G_coef = 3.*self.lengthscale**5/(400*np.sqrt(5))
        Flower = np.array([f(lower) for f in F])[:,None]
        F1lower = np.array([f(lower) for f in F1])[:,None]
        F2lower = np.array([f(lower) for f in F2])[:,None]
        orig = 9./8*np.dot(Flower,Flower.T) + 9.*self.lengthscale**4/200*np.dot(F2lower,F2lower.T)
        orig2 = 3./5*self.lengthscale**2 * ( np.dot(F1lower,F1lower.T) + 1./8*np.dot(Flower,F2lower.T) + 1./8*np.dot(F2lower,Flower.T))
        return(1./self.variance* (G_coef*G + orig + orig2))


In [16]:
kernel = Matern52withDerivatives(input_dim=1, variance=1., lengthscale=1.)

In [20]:
kernel.dK_dr(0.)

array([0.])

In [25]:
def mc(d):
    return (1 + np.sqrt(5)*d + 5*(d*d)) * np.exp(-np.sqrt(5) * d)

def dmc(d):
    return mc(d) * -np.sqrt(5) + (np.sqrt(5) + 10*d)*np.exp(-np.sqrt(5)*d)

def d2mc(d):
    return -np.sqrt(5)*dmc(d) + (np.sqrt(5) + 10*d)*np.exp(-np.sqrt(5) * d)*(-np.sqrt(5)) + np.exp(-np.sqrt(5)*d)*10

In [26]:
d2mc(0.)

4.999999999999999

In [7]:
kernel.dK2_drdr(0.)

array([3.33333333])

In [ ]:
cov_matrix = kernel.K(observations)

In [ ]:
d, N = 1, observations.shape[0]
rows = cols = (d + 1) * N
grad_matrix = np.zeros((rows, cols))

In [ ]:
for i in range(len(observations)):
    for j in range(len(observations)):
        r = np.linalg.norm(observations[i] - observations[j])
        eval_k = kernel.K_of_r(r)
        eval_dk = kernel.dK_dr(r)

In [ ]:
kernel.dK2_drdr(0)